In [1]:
import csv
from decimal import Decimal
import numpy as np
import pandas as pd

一次エネルギー換算係数

9760 kJ/kWh

In [2]:
f_prim = 9760.0

月ごとの日数・始点・終点

In [3]:
d_m = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
d_end = np.cumsum(d_m)
d_str = np.insert(d_end[0: -1], 0, 0)
d_str, d_end

(array([  0,  31,  59,  90, 120, 151, 181, 212, 243, 273, 304, 334],
       dtype=int32),
 array([ 31,  59,  90, 120, 151, 181, 212, 243, 273, 304, 334, 365],
       dtype=int32))

In [4]:
0.3 * 2.4 * 0.5 * 1.1 / 1.0 * 0.001
# kWh/m2

0.00039600000000000003

In [5]:
coeff_v_g = d_m * 0.396 * 0.001 * 24
coeff_v_g

array([0.294624, 0.266112, 0.294624, 0.28512 , 0.294624, 0.28512 ,
       0.294624, 0.294624, 0.28512 , 0.294624, 0.28512 , 0.294624])

局所換気の消費電力をCSVから読み込む。

In [6]:
weekday_raw = []
holiday_out_raw = []
holiday_in_raw = []

with open('ventilation_local.csv') as f:
    reader = csv.reader(f, delimiter='\t')
    rows = [row for row in reader]

    weekday_raw.append(sum([float(row[1]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[2]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[3]) for row in rows[:24]]))
    weekday_raw.append(sum([float(row[4]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[5]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[6]) for row in rows[:24]]))
    weekday_raw.append(sum([float(row[7]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[8]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[9]) for row in rows[:24]]))
    weekday_raw.append(sum([float(row[10]) for row in rows[:24]]))
    holiday_out_raw.append(sum([float(row[11]) for row in rows[:24]]))
    holiday_in_raw.append(sum([float(row[12]) for row in rows[:24]]))

for n in range(4):
    print('平日（' + str(n+1) + '人）: ' + str(weekday_raw[n]))
    print('休日外出（' + str(n+1) + '人）: ' + str(holiday_out_raw[n]))
    print('休日在宅（' + str(n+1) + '人）: ' + str(holiday_in_raw[n]))


平日（1人）: 36.940000000000005
休日外出（1人）: 20.590000000000007
休日在宅（1人）: 40.330000000000005
平日（2人）: 73.81000000000002
休日外出（2人）: 41.02
休日在宅（2人）: 80.59000000000002
平日（3人）: 110.77000000000001
休日外出（3人）: 61.61999999999999
休日在宅（3人）: 120.95
平日（4人）: 147.60999999999999
休日外出（4人）: 82.02000000000001
休日在宅（4人）: 161.18


値を丸める。

In [7]:
weekday = [Decimal(str(v)).quantize(Decimal('0.01')) for v in weekday_raw]
holiday_out = [Decimal(str(v)).quantize(Decimal('0.01')) for v in holiday_out_raw]
holiday_in = [Decimal(str(v)).quantize(Decimal('0.01')) for v in holiday_in_raw]

for n in range(4):
    print('平日（' + str(n+1) + '人）: ' + str(weekday[n]))
    print('休日外出（' + str(n+1) + '人）: ' + str(holiday_out[n]))
    print('休日在宅（' + str(n+1) + '人）: ' + str(holiday_in[n]))

平日（1人）: 36.94
休日外出（1人）: 20.59
休日在宅（1人）: 40.33
平日（2人）: 73.81
休日外出（2人）: 41.02
休日在宅（2人）: 80.59
平日（3人）: 110.77
休日外出（3人）: 61.62
休日在宅（3人）: 120.95
平日（4人）: 147.61
休日外出（4人）: 82.02
休日在宅（4人）: 161.18


年間のスケジュールを読み込む。

In [8]:
vent_schedule = None
with open('schedule.csv', encoding='utf8') as f:
    reader = csv.reader(f, delimiter='\t')
    rows = [row for row in reader]
    vent_schedule = [
        {
            '平日': 'W',
            '休日外': 'HO',
            '休日在': 'HI'
        }[vs[2]] for vs in rows[1:366]
    ]
vent_schedule = np.array(vent_schedule)
print(vent_schedule)

['HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W'
 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W'
 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W'
 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W'
 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'HI' 'W' 'W' 'W' 'HI'
 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W'
 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W'
 'HI' 'HO' 'W' 'W' 'HO' 'HO' 'HI' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO'
 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W'
 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI'
 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W'
 'W' 'W' 'W' 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'HI' 'W'
 'HI' 'HO' 'W' 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 'W' 'HI' 'HO' 'W'
 'W' 'W' 'W' 'W' 'HI' 'HI' 'W' 'W' 'W' 'W' 

日付dの時間tにおける局所換気の消費電力を整理する。

In [9]:
e_e_vl_1_d_t = np.zeros(shape=365, dtype=float)
e_e_vl_2_d_t = np.zeros(shape=365, dtype=float)
e_e_vl_3_d_t = np.zeros(shape=365, dtype=float)
e_e_vl_4_d_t = np.zeros(shape=365, dtype=float)

e_e_vl_1_d_t[vent_schedule=='W'] = weekday[0]
e_e_vl_1_d_t[vent_schedule=='HI'] = holiday_in[0]
e_e_vl_1_d_t[vent_schedule=='HO'] = holiday_out[0]

e_e_vl_2_d_t[vent_schedule=='W'] = weekday[1]
e_e_vl_2_d_t[vent_schedule=='HI'] = holiday_in[1]
e_e_vl_2_d_t[vent_schedule=='HO'] = holiday_out[1]

e_e_vl_3_d_t[vent_schedule=='W'] = weekday[2]
e_e_vl_3_d_t[vent_schedule=='HI'] = holiday_in[2]
e_e_vl_3_d_t[vent_schedule=='HO'] = holiday_out[2]

e_e_vl_4_d_t[vent_schedule=='W'] = weekday[3]
e_e_vl_4_d_t[vent_schedule=='HI'] = holiday_in[3]
e_e_vl_4_d_t[vent_schedule=='HO'] = holiday_out[3]

月ごとに積算する。

In [10]:
e_e_vl_1_m = [np.around(e_e_vl_1_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]
e_e_vl_2_m = [np.around(e_e_vl_2_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]
e_e_vl_3_m = [np.around(e_e_vl_3_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]
e_e_vl_4_m = [np.around(e_e_vl_4_d_t[d_s: d_e].sum(), 2) for (d_s, d_e) in zip(d_str, d_end)]

In [11]:
for m in range(12):
    print(str(m) + '月 1人=' + str(e_e_vl_1_m[m]) + ', 2人=' + str(e_e_vl_2_m[m]) + ', 3人=' + str(e_e_vl_3_m[m]) + ', 4人=' + str(e_e_vl_4_m[m]))

0月 1人=1136.17, 2人=2269.99, 3人=3406.83, 4人=4539.72
1月 1人=1021.96, 2人=2041.78, 3人=3064.34, 4人=4083.32
2月 1人=1136.17, 2人=2269.99, 3人=3406.83, 4人=4539.72
3月 1人=1082.88, 2人=2163.39, 3人=3246.91, 4人=4326.52
4月 1人=1103.47, 2人=2204.41, 3人=3308.53, 4人=4408.54
5月 1人=1095.84, 2人=2189.4, 3人=3285.88, 4人=4378.54
6月 1人=1142.95, 2人=2283.55, 3人=3427.19, 4人=4566.86
7月 1人=1132.78, 2人=2263.21, 3人=3396.65, 4人=4526.15
8月 1人=1082.88, 2人=2163.39, 3人=3246.91, 4人=4326.52
9月 1人=1119.82, 2人=2237.2, 3人=3357.68, 4人=4474.13
10月 1人=1102.62, 2人=2202.96, 3人=3306.24, 4人=4405.68
11月 1人=1119.82, 2人=2237.2, 3人=3357.68, 4人=4474.13


In [12]:
e_e_vl_m = np.array([e_e_vl_1_m, e_e_vl_2_m, e_e_vl_3_m, e_e_vl_4_m]).T
e_e_vl_m

array([[1136.17, 2269.99, 3406.83, 4539.72],
       [1021.96, 2041.78, 3064.34, 4083.32],
       [1136.17, 2269.99, 3406.83, 4539.72],
       [1082.88, 2163.39, 3246.91, 4326.52],
       [1103.47, 2204.41, 3308.53, 4408.54],
       [1095.84, 2189.4 , 3285.88, 4378.54],
       [1142.95, 2283.55, 3427.19, 4566.86],
       [1132.78, 2263.21, 3396.65, 4526.15],
       [1082.88, 2163.39, 3246.91, 4326.52],
       [1119.82, 2237.2 , 3357.68, 4474.13],
       [1102.62, 2202.96, 3306.24, 4405.68],
       [1119.82, 2237.2 , 3357.68, 4474.13]])

係数 $\alpha$

MJ/m2

In [13]:
alpha = coeff_v_g * f_prim * 0.001
alpha

array([2.87553024, 2.59725312, 2.87553024, 2.7827712 , 2.87553024,
       2.7827712 , 2.87553024, 2.87553024, 2.7827712 , 2.87553024,
       2.7827712 , 2.87553024])

係数 $\beta$

MJ

In [14]:
beta = e_e_vl_m * 0.001 * f_prim * 0.001
beta

array([[11.0890192, 22.1551024, 33.2506608, 44.3076672],
       [ 9.9743296, 19.9277728, 29.9079584, 39.8532032],
       [11.0890192, 22.1551024, 33.2506608, 44.3076672],
       [10.5689088, 21.1146864, 31.6898416, 42.2268352],
       [10.7698672, 21.5150416, 32.2912528, 43.0273504],
       [10.6953984, 21.368544 , 32.0701888, 42.7345504],
       [11.155192 , 22.287448 , 33.4493744, 44.5725536],
       [11.0559328, 22.0889296, 33.151304 , 44.175224 ],
       [10.5689088, 21.1146864, 31.6898416, 42.2268352],
       [10.9294432, 21.835072 , 32.7709568, 43.6675088],
       [10.7615712, 21.5008896, 32.2689024, 42.9994368],
       [10.9294432, 21.835072 , 32.7709568, 43.6675088]])

In [15]:
with pd.ExcelWriter('vent_table.xlsx') as writer:
    pd.DataFrame(
        coeff_v_g,
        index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月'],
        columns=['月別の係数, kWh/m2']
    ).to_excel(writer, sheet_name='table_1')
    pd.DataFrame(
        [weekday, holiday_out, holiday_in],
        index=['平日', '休日外出', '休日在宅'],
        columns=['1人','2人','3人','4人']
    ).to_excel(writer, sheet_name='table_2')
    pd.DataFrame(
        e_e_vl_m,
        index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月'],
        columns=['1人','2人','3人','4人']
    ).T.to_excel(writer, sheet_name='table_3')
    pd.DataFrame(
        alpha,
        index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月'],
        columns=['係数alpha, MJ/m2']
    ).to_excel(writer, sheet_name='table_4')
    pd.DataFrame(
        beta,
        index=['1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月','12月'],
        columns=['1人','2人','3人','4人']
    ).to_excel(writer, sheet_name='table_5')

